# Machine learning for free energy functional for 1D Lennard Jones fluid
## -Traning part
***
## -square+Cubic term
## This takes quite long, maybe days-weeks on 8 CPU depend on parameters 
### Train $F^{ML}=\epsilon \beta n_i n_j +\epsilon \beta_2 n_i^{\prime} n_j^{\prime} n_k^{\prime} $
### in paper, train sq term  and cubic separate, then add together with noise then train 
### The idea is that cubic terms are more like correction terms 
### no guarantee this will work

In [1]:
import numpy as np
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import pymp
num_thread = 8

from numba import jit
from timeit import default_timer as timer

In [2]:
file_dir = "fuzzy"

In [3]:
f = open('LJ_data_'+file_dir+'/MC_parameter.dat', 'r')
temp = f.read().split("\t")
L=float(temp[0])
dx=float(temp[1])
N=int(L/dx)
batch_size = int(temp[2])
f.close()
temp 

['32', '0.03125', '256\n']

In [4]:
print(str(dx)+"\t"+str(N)+"\t"+str(L)+"\t"+str(batch_size))

0.03125	1024	32.0	256


In [5]:
mini_batch = 16

In [6]:
f = open('LJ_data_'+file_dir+'/MC_inform.dat', 'r')
MC_inform = f.read().splitlines()
for i in range(len(MC_inform)):
    #print(MC_inform[i])
    MC_inform[i]=MC_inform[i].split("\t")
f.close()
batch_size = (len(MC_inform)//mini_batch)*mini_batch

In [7]:
print(str(dx)+"\t"+str(N)+"\t"+str(L)+"\t"+str(batch_size))

0.03125	1024	32.0	256


In [8]:
MC_inform = np.array(MC_inform)
MC_inform = MC_inform.astype(float)

In [9]:
Nw = 8
Lw = int(np.floor(8/dx)+1)
LLw = int(np.floor(8/dx)/2)
rho_array = np.zeros((mini_batch,N))
rho0_array = np.zeros((mini_batch))
epsilon_array = np.zeros((mini_batch))
mu_array = np.zeros((mini_batch))
Vext_array = np.zeros((mini_batch,N))
Lw*dx,LLw

(8.03125, 128)

In [10]:
def symmetry (epsilon):
    for i in range(Nw):
        for j in range(i,Nw):
            for k in range(j,Nw):
                temp = epsilon[i][j][k]
                epsilon[i][k][j]=temp
                epsilon[j][i][k]=temp
                epsilon[j][k][i]=temp
                epsilon[k][i][j]=temp
                epsilon[k][j][i]=temp
    return epsilon

In [11]:
"""initial w"""
def initial_kernel():
    
    np.random.seed(0)
    w1 = np.random.normal(0,1,Nw*Lw)*0.01
    w1=w1.reshape((Nw,Lw))
    
    np.random.seed(1)
    w2 = np.random.normal(0,1,Nw*Lw)*0.01
    w2=w2.reshape((Nw,Lw))
       
    
    np.random.seed(2)
    beta1 = np.random.normal(0,1,Nw**2)*0.01
    beta1=beta1.reshape((Nw,Nw))
    beta1 = ((beta1)+beta1.T)/2
    
    np.random.seed(3)
    beta2 = np.random.normal(0,1,Nw**3)*0.01
    beta2=  beta2.reshape((Nw,Nw,Nw))
    beta2 = symmetry(beta2)
    
    return w1,w2,beta1,beta2

In [12]:
w1,w2,beta1,beta2 = initial_kernel()
print(beta1[0][2],beta1[2][0])
print(beta2[0][2][1],beta2[2][1][0],beta2[1][2][0],beta2[1][0][2],beta2[2][0][1],beta2[2][1][0])

-0.010776632960948027 -0.010776632960948027
-0.013138647533626821 -0.013138647533626821 -0.013138647533626821 -0.013138647533626821 -0.013138647533626821 -0.013138647533626821


In [13]:
MC_inform[32][0]

32.0

In [14]:
"""load data with shuffle"""
def load_data (batch_number):
    for j in range(mini_batch):
        i = batch_number[j]
        
        rho_array[j]=np.loadtxt("LJ_data_"+file_dir+"/rho_"+str(i)+".dat")
        #rho_array[j]=(rho_array[j]+rho_array[j][::-1])/2 #force symmetry ? 
        
        Vext_array[j]=np.loadtxt("LJ_data_"+file_dir+"/Vext_"+str(i)+".dat")
        epsilon_array[j]=float(MC_inform[i][1])
        mu_array[j]=np.log(float(MC_inform[i][2]))
        
    return rho_array,Vext_array,epsilon_array,mu_array

In [15]:
@jit
def conv (rho,w):

    rho_pad=np.pad(rho,(LLw,LLw),'symmetric')
    rho_pad[0:LLw]=rho_pad[N:N+LLw]
    rho_pad[N+LLw:N+LLw*2]=rho_pad[LLw:LLw+LLw]
    n = np.correlate(rho_pad,w)
    
    return n*dx

In [16]:
@jit
def w_FMT(eps):
    #d=cal_eff_diameter (eps)
    R = 1.0/2
    k=np.linspace(0,N//2,N//2+1)*2*np.pi/L
    w0=2*np.cos(k*R)/2
    k[0]=1 #keep notebook shutup
    w1=2*np.sin(k*R)/k
    w1[0]=2*R
    return w0,w1;

w0_FMT,w1_FMT = w_FMT(0)

def cal_n0(rho):
    return np.fft.irfft(np.fft.rfft(rho)*w0_FMT)
def cal_n1(rho):
    return np.fft.irfft(np.fft.rfft(rho)*w1_FMT)

def cal_c1_FMT(rho,eps):
    n0=cal_n0(rho)
    n1=cal_n1(rho)
    F0=-np.log(1-n1)
    F1=n0/(1-n1)
    return cal_n0(F0)+cal_n1(F1)
    
    

In [17]:
"""add FMT terms"""
@jit
def cal_c1(epsilon,beta1,beta2,n,nnw,w1,w2,rho):
    c1 = np.zeros((N))
       
    for i in range(Nw):
        for j in range(Nw):
            c1+=epsilon*beta1[i][j]*(conv(n[i],w1[j])+conv(n[j],w1[i]))
            
    for i in range(Nw):
        for j in range(Nw):
            for k in range(Nw):
                c1+=(epsilon*beta2[i][j][k])*(
                    nnw[i][j][k]+nnw[i][k][j]+nnw[j][k][i])
    c1+=cal_c1_FMT(rho,epsilon)
    return c1

In [18]:
@jit
def cal_mu_ML_test(epsilon,beta1,beta2,rho0,w1,w2):
    c1=0
    
    for i in range(Nw):
        for j in range(Nw):
                ni = np.sum(rho0*w1[i])
                nj = np.sum(rho0*w1[j])
                c1+=(epsilon*beta1[i][j])*(
                    np.sum(nj*w1[i])+np.sum(ni*w1[j]))*dx*dx
    
    
    for i in range(Nw):
        for j in range(Nw):
            for k in range(Nw):
                ni = np.sum(rho0*w2[i])
                nj = np.sum(rho0*w2[j])
                nk = np.sum(rho0*w2[k])
                c1+=(epsilon*beta2[i][j][k])*(
                    np.sum(ni*nj*w2[k])+np.sum(nj*nk*w2[i])+np.sum(nk*ni*w2[j]))*dx*dx*dx
    rho_temp = np.full(N,rho0)
    c1_FMT = cal_c1_FMT(rho_temp,epsilon)[0]
    return c1+np.log(rho0)+c1_FMT


In [19]:
@jit
def f2 (c1,c2,x,mu,eps):
    rho_temp = np.full(N,x)
    c1_FMT = cal_c1_FMT(rho_temp,eps)[0]
    return (c1*x+c2*x**2+np.log(x)+c1_FMT-mu)**2

@jit
def cal_rho0(mu,beta1,beta2,epsilon,w1,w2,mu_input):
    
    c1=0  
    c2=0
    
    for i in range(Nw):
        for j in range(Nw):
            c1+=epsilon*beta1[i][j]*(
                np.sum(np.sum(w1[i])*w1[j]))*dx**2*2
    
    
    for i in range(Nw):
        for j in range(Nw):
            for k in range(Nw):
                ni = np.sum(w2[i])
                nj = np.sum(w2[j])
                nk = np.sum(w2[k])
                c2+=(epsilon*beta2[i][j][k])*(
                    np.sum(ni*nj*w2[k])+np.sum(nj*nk*w2[i])+
                    np.sum(nk*ni*w2[j]))*dx*dx*dx
    
    rho_guess = np.zeros(3)
    res = np.zeros(3)
    
    interval = 0.01

    for i in range(3):
        rho_guess[i] = 0.3+interval*i
        
    for iteration in range (10000):
        for i in range(3):
            res[i] = f2(c1,c2,rho_guess[i],mu,epsilon)
        if(res[0]>res[1] and res[2]>res[1]):
            interval/=2
            rho_guess[0]=rho_guess[1]-interval
            rho_guess[2]=rho_guess[1]+interval
        elif(res[1]>res[0] and res[2]>res[0]):
            temp = rho_guess[0]
            rho_guess[0]=temp-interval
            rho_guess[1]=temp
            rho_guess[2]=temp+interval
        else:
            temp = rho_guess[2]
            rho_guess[0]=temp-interval
            rho_guess[1]=temp
            rho_guess[2]=temp+interval
        if(interval<10**-8):
            break        
        for i in range (3):
            if(rho_guess[i]<=0 or rho_guess[i]>=1):
                break
    temp=rho_guess[1]
    #print("rho\t=\t",temp,"\t z input=",np.exp(mu_input), "\t z guess=",np.exp(mu),"\t z predict=",np.exp(cal_mu_ML_test(epsilon,beta1,beta2,temp,w1,w2)))
    if(temp<0):
        return 0.01
    elif(temp>1):
        return 0.9
    else:
        return rho_guess[1]

In [20]:

rho0=cal_rho0(np.log(1.5),beta1,beta2,0.5,w1,w2,np.log(1.5))

print(rho0)

0.42057922363281264


In [21]:
@jit
def f (rho,c,rhoML):
    return np.sum((rho-c*rhoML)**2)

@jit
def cal_mu_ML(rho,c1,Vext):
    rhoML = np.exp(-c1-Vext)
    mu_guess = np.zeros(3)
    res = np.zeros(3)
    
    for i in range(3):
        mu_guess[i] = i-1.0
    interval = 1
    
    for iteration in range (1000):
        for i in range(3):
            res[i] = f(rho,mu_guess[i],rhoML)
        if(res[0]>res[1] and res[2]>res[1]):
            interval/=2
            mu_guess[0]=mu_guess[1]-interval
            mu_guess[2]=mu_guess[1]+interval
        elif(res[1]>res[0] and res[2]>res[0]):
            temp = mu_guess[0]
            mu_guess[0]=temp-interval
            mu_guess[1]=temp
            mu_guess[2]=temp+interval
        else:
            temp = mu_guess[2]
            mu_guess[0]=temp-interval
            mu_guess[1]=temp
            mu_guess[2]=temp+interval
        if(interval<10**-8):
            break
    
    return np.log(mu_guess[1])
            
    
     

In [22]:
@jit
def cal_rho_ML(mu,c1,V):
    rho_ML = np.exp(-c1-V+mu)
    return rho_ML

In [23]:
@jit
def error (rho,rho_ML):
    return np.sum((rho-rho_ML)**2)*dx


In [24]:
@jit
def Dbeta1_tot(rho_ML,rho,rho0,eps,n,w,i,j):
    temp=np.zeros((N))    
    for k in range(mini_batch):
        c1=(np.sum(np.sum(rho0[k]*w[i])*w[j]))*2*dx**2  
        temp+=(2.0)*(rho_ML[k]-rho[k])*rho_ML[k]*eps[k]*(c1-conv(n[k][i],w[j])-conv(n[k][j],w[i]))
    
    return np.sum(temp)*dx

In [25]:
"""Check epsilon"""
@jit
def Dbeta2_tot(rho_ML,rho,rho0,epsilon,nnw,w,i,j,k):
    temp1=np.zeros((N))
    temp2=np.zeros((N))
    for MM in range(mini_batch):
        ni = np.sum(rho0[MM]*w[i])
        nj = np.sum(rho0[MM]*w[j])
        nk = np.sum(rho0[MM]*w[k])
        c1=(np.sum(ni*nj*w[k])+np.sum(nj*nk*w[i])+np.sum(nk*ni*w[j]))*dx*dx*dx
        temp = (2.0)*(rho_ML[MM]-rho[MM])*rho_ML[MM]*(c1-nnw[MM][i][j][k]-nnw[MM][j][k][i]-nnw[MM][i][k][j])
        temp1+=temp*epsilon[MM]
        #temp2+=temp*epsilon[MM]**2
    return np.sum(temp1)*dx

In [26]:
@jit
def Dw1F(rhow,beta,j):
    temp=np.zeros((N))
    for i in range(Nw):
        temp+=rhow[i]*beta[i][j]
    return temp
@jit
def Dw1mu(beta,rho0,w,j):
    c1=0
    for i in range(Nw):
        c1+=4*(beta[i][j])*(np.sum(rho0*w[i]))*dx**2
    return c1

@jit
def Dw1_tot(rho_ML,rho,rho0,rhow,j,p,beta,eps,w):
    temp=np.zeros(mini_batch)
    for k in range(mini_batch): 
        temp[k]=np.sum((rho_ML[k]-rho[k])*(2)*rho_ML[k]*eps[k]*(
            Dw1mu(beta,rho0[k],w,j)-Dw1F(rhow[k][p],beta,j)))
    #print(temp)
    return np.sum(temp)*dx

In [27]:
@jit
def Dw2F(n,beta2,epsilon,k,p,w,rho,nij):
    temp=np.zeros((N))
    rho_new = np.roll(rho,-(p-LLw))
    n_temp = np.zeros((Nw,N))
    niconvj = np.zeros((Nw,Nw,N))     
    
    for i in range(Nw):
        n_temp[i] = n[i]*rho_new
    
    for i in range(Nw):
        for j in range(Nw):
            niconvj[i][j] = conv(n_temp[i],w[j])
     
    
    for i in range(Nw):
        for j in range(Nw):
            temp+=3*(epsilon*beta2[i][j][k])*np.roll(nij[i][j],-(p-LLw))*dx
            temp+=3*(epsilon*beta2[i][j][k])*niconvj[i][j]*dx
            temp+=3*(epsilon*beta2[i][j][k])*niconvj[j][i]*dx
            
    return temp

In [28]:
@jit
def Dw2mu(beta2,epsilon,rho0,w,k):
    c1=0
        
    for i in range(Nw):
        ni=np.sum(rho0*w[i])
        for j in range(Nw):
            nj=np.sum(rho0*w[j])
            c1+=9*(epsilon*beta2[i][j][k])*(ni*nj)*dx**3
    return c1

In [29]:
@jit
def Dw2_tot(rho_ML,rho,rho0,n,k,p,beta2,epsilon,w,nij):
    temp=np.zeros(mini_batch)
    for M in range(mini_batch): 
        temp[M]=np.sum((rho_ML[M]-rho[M])*(2)*rho_ML[M]*(Dw2mu(beta2,epsilon[M],rho0[M],w,k)-
                                                         Dw2F(n[M],beta2,epsilon[M],
                                                             k,p,w,rho[M],nij[M])))
                                                                             
    #print(temp)
    return np.sum(temp)*dx

In [30]:
@jit
def kernel_rhow(n_array,w):
    rhow = np.zeros((Lw,Nw,N))
    for k in range(Nw):
        for p in range(Lw):
            rhow[p][k]=np.roll(n_array[k],-(p-LLw))*dx*4
    return rhow

In [31]:

def ML_kernel(seed,w1,w2,beta1,beta2,alpha):
    np.random.seed(seed)
    batch_number=np.arange(batch_size)
    np.random.shuffle(batch_number)
    
    dbeta1 = pymp.shared.array((beta1.shape))
    dbeta2 = pymp.shared.array((beta2.shape))
    dw1 = pymp.shared.array((w1.shape))
    dw2 = pymp.shared.array((w2.shape))
    
    rhoML_array = pymp.shared.array((mini_batch,N))
    n1_array = pymp.shared.array((mini_batch,Nw,N))
    n2_array = pymp.shared.array((mini_batch,Nw,N))
    
    rhow = pymp.shared.array((mini_batch,Lw,Nw,N), dtype='float')

    nnw = pymp.shared.array((mini_batch,Nw,Nw,Nw,N))
    #nw = pymp.shared.array((mini_batch,Nw,Nw,N))
    nij = pymp.shared.array((mini_batch,Nw,Nw,N))

    epco_error = 0
    N_minbatch = int(batch_size/mini_batch)
    for M in range(N_minbatch):
        print(str(M)+'/'+str(N_minbatch))
        rho_array,Vext_array,epsilon_array,mu_array=load_data (batch_number[M*mini_batch:(M+1)*mini_batch])
        
        '''preparing n*w'''
        
        with pymp.Parallel(num_thread) as p:
            for MM in p.range(mini_batch):
                for j in range (Nw):
                    n1_array[MM][j] = conv(rho_array[MM],w1[j])
                    n2_array[MM][j] = conv(rho_array[MM],w2[j])
                    

        with pymp.Parallel(num_thread) as p:            
            for MM in p.range(mini_batch):
                for i in range(Nw):
                    for j in range(Nw):
                        nij[MM][i][j]=n2_array[MM][i]*n2_array[MM][j] 
        '''               
        with pymp.Parallel(num_thread) as p:            
            for MM in p.range(mini_batch):
                for i in range(Nw):
                    for j in range(Nw):
                        nw[MM][i][j]=conv(n2_array[MM][i],w[j]) 
        '''            
        with pymp.Parallel(num_thread) as p:            
            for MM in p.range(mini_batch):
                for i in range(Nw):
                    for j in range(Nw):
                        for k in range(Nw):
                            nnw[MM][i][j][k]=conv(nij[MM][i][j],w2[k])
                    
        with pymp.Parallel(num_thread) as p:            
            for MM in p.range(mini_batch):
                c1 = cal_c1(epsilon_array[MM],beta1,beta2
                            ,n1_array[MM],nnw[MM],w1,w2,rho_array[MM])
                #print(c1.shape)
                mu_ML = cal_mu_ML(rho_array[MM],c1,Vext_array[MM])
                #print("z_input",np.exp(mu_array[MM]))
                rho0_array[MM]=cal_rho0(mu_ML,beta1,beta2,
                                        epsilon_array[MM],w1,w2,mu_array[MM])
                rhoML_array[MM] = cal_rho_ML(mu_ML,c1,Vext_array[MM])

        epco_error+=error(rho_array,rhoML_array)
        
        with pymp.Parallel(num_thread) as p:
            for i in p.range(Nw):
                for j in range(i,Nw):
                    dbeta1[i][j] =Dbeta1_tot(
                        rhoML_array,rho_array,
                        rho0_array,epsilon_array,n1_array,w1,
                        i,j)
                    dbeta1[j][i]=dbeta1[i][j]
        
        with pymp.Parallel(num_thread) as p:
            for i in p.range(Nw):
                for j in range(i,Nw):
                    for k in range(j,Nw):
                        dbeta2[i][j][k] =Dbeta2_tot(
                            rhoML_array,rho_array,
                            rho0_array,epsilon_array,nnw,w2,i,j,k)
                        dbeta2[i][k][j]=dbeta2[i][j][k] 
                        dbeta2[j][i][k]=dbeta2[i][j][k] 
                        dbeta2[j][k][i]=dbeta2[i][j][k] 
                        dbeta2[k][i][j]=dbeta2[i][j][k] 
                        dbeta2[k][j][i]=dbeta2[i][j][k]             
        

        """
        with pymp.Parallel(num_thread) as p:            
            for MM in p.range(mini_batch):
                rhow[MM]=kernel_rhow(n_array[MM],w)
        """
        
        #Dw2_tot(rho_ML,rho,rho0,n,k,p,beta2,epsilon,w,nij):
        
        with pymp.Parallel(num_thread) as p:            
            for i in p.range(mini_batch):
                rhow[i]=kernel_rhow(n1_array[i],w1)
        
        with pymp.Parallel(num_thread) as pp:            
            for j in pp.range(Nw):
                for p in range(Lw):
                    dw1[j][p]=Dw1_tot(
                        rhoML_array,rho_array,rho0_array,rhow,
                        j,p,beta1,epsilon_array,w1)
        
        with pymp.Parallel(num_thread) as pp:            
            for j in pp.range(Nw):
                for p in range(Lw):
                    dw2[j][p]=Dw2_tot(rhoML_array,rho_array,
                                    rho0_array,n2_array,j,p,
                                    beta2,epsilon_array,w2,nij)
                    
                    
               
        w1-=alpha*dw1*dx
        beta1-=alpha*dbeta1*dx*dx
        w2-=alpha*dw2
        beta2-=alpha*dbeta2*dx
        
    return w1,w2,beta1,beta2,epco_error

In [32]:
def save_data(w1,w2,beta1,beta2,error_array):
    """save kernel results"""
    np.savetxt(r"./ML_data/epco_error_sq_cubic.txt",error_array)
    np.savetxt(r"./ML_data/w1_kernel_sq_cubic.txt",w1)
    np.savetxt(r"./ML_data/w2_kernel_sq_cubic.txt",w2)
    np.savetxt(r"./ML_data/beta1_kernel_sq_cubic.txt",beta1)
        
    for i in range(Nw):
        np.savetxt(r"./ML_data/beta2_kernel_sq_cubic_"+str(i)+".txt",beta2[i])
        
    f = open('./ML_data/parameter_kernel_sq_cubic.txt', 'w')
    f.write(str(dx))
    f.write("\n")
    f.write(str(Lw))
    f.write("\n")
    f.write(str(Nw))
    f.write("\n")
    f.close()
    """save done"""

In [33]:
np.random.seed(10)
w1 = np.zeros(Nw*Lw)
w2 = np.zeros(Nw*Lw)

beta1 = np.zeros(Nw**2)
beta2 = np.zeros(Nw**3)
w1,w2,beta1,beta2=initial_kernel()

error_array=np.empty(0)
alpha_array=np.empty(0)
start_time = timer()

#loadfile####

w1=np.loadtxt("./ML_data/w1_kernel_sq_cubic.txt")
temp=np.random.normal(0,1,w1.shape)*0.05
w1+=temp

w2=np.loadtxt("./ML_data/w_kernel_sq_cubic.txt")
temp=np.random.normal(0,1,w2.shape)*0.05
w2+=temp

beta1=np.loadtxt("./ML_data/beta1_kernel_sq_cubic.txt")
temp=np.random.normal(0,1,beta1.shape)*0.05
beta1+=temp

for i in range(Nw):
    beta2[i]=np.loadtxt("./ML_data/beta2_sq_cubic"+str(i)+".txt")
temp=np.random.normal(0,1,beta2.shape)*0.05
beta2+=temp
    
#print(w1.shape,w2.shape,beta1.shape,beta2.shape)
#############
save_data(w1,w2,beta1,beta2,error_array)
alpha = 0.01
for i in range(100000):
    w1_old = np.copy(w1)
    w2_old = np.copy(w2)
    beta1_old = np.copy(beta1)
    beta2_old = np.copy(beta2)
    w1,w2,beta1,beta2,epco_error=ML_kernel(i,w1,w2,beta1,beta2,alpha)
    error_array=np.append(error_array, epco_error)
    alpha_array=np.append(alpha_array, alpha)
    
    if(alpha<10**-6 or np.isnan(epco_error)):
        break
        
    if(epco_error>error_array[i-1] and i>0):
        alpha/=2.0
        print("alpha=",alpha)
    
    print(i,epco_error,alpha)
    save_data(w1,w2,beta1,beta2,error_array)
end_time = timer()
print("elapse="+'\t'+str(end_time-start_time))
save_data(w1,w2,beta1,beta2,error_array)


0/16
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
0 10.840465462230163 0.01
0/16
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
1 9.996028286572757 0.01
0/16
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
2 9.163590230826754 0.01
0/16
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
3 8.33733137792191 0.01
0/16
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
4 7.5292215199767725 0.01
0/16
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
5 6.755954216531431 0.01
0/16
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
6 6.0335313645505355 0.01
0/16
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
7 5.376918242906097 0.01
0/16
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
8 4.800346791682026 0.01

1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
73 2.4312291376485957 0.01
0/16
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
74 2.4280210337313504 0.01
0/16
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
75 2.424040195128082 0.01
0/16
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
76 2.4204822859598427 0.01
0/16
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
77 2.4167610651323614 0.01
0/16
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
78 2.4135143367839498 0.01
0/16
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
79 2.4093383607754446 0.01
0/16
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
80 2.4068498686151187 0.01
0/16
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
81 2.40279527340

ConnectionResetError: [Errno 104] Connection reset by peer